In [2]:
from sentinelhub import WebFeatureService, BBox, CRS, DataSource, SHConfig
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
import pandas as pd
from collections import OrderedDict
from sentinelhub import AwsTileRequest
from env_vars import sentinel_username,sentinel_password

In [3]:
# # full_congo_basin_geo = './data/Geometry/congo_basin_boundary/congo_basin_boundary_custom_v3.geojson'
# republic_of_the_congo_geo = '/Volumes/Lacie/zhenyadata/Project_Canopy_Data/Data/Geometry/republic_of_the_congo/republic_of_the_congo_boundary_simplified.geojson'

# # Query Products

# api = SentinelAPI(sentinel_username, sentinel_password, "https://scihub.copernicus.eu/apihub/")


# # download single scene by known product id
# # api.download("3f09e58e-b893-46fa-ad05-6bf1f5b4b257")

# # search by polygon, time, and SciHub query keywords
# footprint = geojson_to_wkt(read_geojson(republic_of_the_congo_geo))
# products = api.query(footprint,
#                      date=('20200101', date(2020, 8, 10)),
#                      area_relation = "IsWithin",
#                      raw='S2MSI2A',
#                      platformname='Sentinel-2',
#                      cloudcoverpercentage=(0, 5))


# # # # convert to Pandas DataFrame
# products_df = api.to_dataframe(products)

In [4]:
INSTANCE_ID = 'b6591791-2e5b-4502-b9bf-772b3a1be18b'  # In case you put instance ID into configuration file you can leave this unchanged

if INSTANCE_ID:
    config = SHConfig()
    config.instance_id = INSTANCE_ID
else:
    config = None

In [5]:
# sample code

# wms_bands_request_2017 = WmsRequest(
#     layer='BANDS-S2-L2A',
#     bbox=crown_bbox, 
#     time=('2017-03-01','2017-11-30'),
#     config=config,
#     data_folder = 'C:\\User\\BANDS-S2-L2A\\2017'
# )

In [6]:
# #Congo Basin Bounding Box
# # search_bbox = BBox(bbox=[6.81,-5.27, 30, 5.5], crs=CRS.WGS84)

# #Additional Params
# bands = ['R10m/TCI']

# #Republic of the Congo Bounding Box
# search_bbox = BBox(bbox=[11.6395900000061374,-4.8205900000029942, 18.4909000000043306,3.6516899999987800], crs=CRS.WGS84)


# j = [2020]
# def annual_results():
#     for i in j: 
#         search_time_interval = (f'{i}-01-01T00:00:00', f'{i}-12-31T23:59:59')
#         wfs_iterator = WebFeatureService(
#             search_bbox,
#             search_time_interval,
#             data_source=DataSource.SENTINEL2_L2A,
#             maxcc=.05,
#             config=config
#         )
#         results = wfs_iterator.get_tiles()
#         df = pd.DataFrame(results, columns=['Tilename','Date','AmazonID'])
#         df2 = df.drop_duplicates(subset='Tilename')
#         output2 = list(df2.itertuples(index=False,name=None))
#         for tile in output2:
#             tile_name, time, aws_index = tile
#             data_folder = f'/Volumes/Lacie/zhenyadata/Project_Canopy_Data/Data/Sentinel_Data/ROC/L2A/{i}'
#             request = AwsTileRequest(
#                 tile=tile_name,
#                 time=time,
#                 bands = bands, 
#                 aws_index=aws_index,
#                 data_folder=data_folder,
#                 data_source=DataSource.SENTINEL2_L2A,
#                 safe_format = True
#             )

#             request.save_data()
        

# Download All Tiles (Including Duplicates)

In [259]:
#Congo Basin Bounding Box
# search_bbox = BBox(bbox=[6.81,-5.27, 30, 5.5], crs=CRS.WGS84)

#Additional Params
bands = ['R10m/TCI']

#Republic of the Congo Bounding Box
search_bbox = BBox(bbox=[11.6395900000061374,-4.8205900000029942, 18.4909000000043306,3.6516899999987800], crs=CRS.WGS84)



search_time_interval = ('2020-03-01T00:00:00', '2020-12-31T23:59:59')
wfs_iterator = WebFeatureService(
    search_bbox,
    search_time_interval,
    data_source=DataSource.SENTINEL2_L2A,
    maxcc=.05,
    config=config
)
results = wfs_iterator.get_tiles()
df = pd.DataFrame(results, columns=['Tilename','Date','AmazonID'])
output2 = list(df.itertuples(index=False,name=None))
for tile in output2:
    tile_name, time, aws_index = tile
    data_folder = f'/Volumes/Lacie/zhenyadata/Project_Canopy_Data/Data/Sentinel_Data/ROC/L2A/SAFE/Full/2020'
    request = AwsTileRequest(
        tile=tile_name,
        time=time,
        bands = bands, 
        aws_index=aws_index,
        data_folder=data_folder,
        data_source=DataSource.SENTINEL2_L2A,
        safe_format = True
    )

    request.save_data(redownload=True)
        

/Users/user/.local/share/virtualenvs/data_collection-6QAt_VD6/lib/python3.7/site-packages/sentinelhub/aws.py:164: SHUserWarning: Products with baseline 02.14 are not officially supported in sentinelhub-py. If you notice any errors in naming structure of downloaded data please report an issue at https://github.com/sentinel-hub/sentinelhub-py/issues. Pull requests are also very appreciated
  warnings.warn(message, category=SHUserWarning)
/Users/user/.local/share/virtualenvs/data_collection-6QAt_VD6/lib/python3.7/site-packages/sentinelhub/aws.py:164: SHUserWarning: Products with baseline 02.14 are not officially supported in sentinelhub-py. If you notice any errors in naming structure of downloaded data please report an issue at https://github.com/sentinel-hub/sentinelhub-py/issues. Pull requests are also very appreciated
  warnings.warn(message, category=SHUserWarning)
/Users/user/.local/share/virtualenvs/data_collection-6QAt_VD6/lib/python3.7/site-packages/sentinelhub/aws.py:164: SHUser

# Duplicate tile analysis

In [63]:
#Additional Params
bands = ['R10m/TCI']

#Republic of the Congo Bounding Box
search_bbox = BBox(bbox=[11.6395900000061374,-4.8205900000029942, 18.4909000000043306,3.6516899999987800], crs=CRS.WGS84)


def detect_duplicates(start_years):
    for result in start_years:
        search_time_interval = (f'{result}-01-01T00:00:00', '2020-12-31T23:59:59')
        wfs_iterator = WebFeatureService(
            search_bbox,
            search_time_interval,
            data_source=DataSource.SENTINEL2_L2A,
            maxcc=.05,
            config=config
        )
        results = wfs_iterator.get_tiles()
        df = pd.DataFrame(results, columns=['Tilename','Date','AmazonID'])
        print("Between",result, "and August 2020")
        print("Total tiles:",len(df))
        print("Total unique tiles:",len(df.drop_duplicates(subset='Tilename')))
        print("% Tiles < 10 duplicate:", (df["Tilename"].value_counts() < 10).value_counts()[1] / len(df.drop_duplicates(subset='Tilename')))
        print("---------------------")
        print("\n")

In [64]:
start_year = []
for i in range(2017,2021,1):
    start_year.append(i)

In [66]:
detect_duplicates(start_year)

Between 2017 and August 2020
Total tiles: 4319
Total unique tiles: 107
% Tiles < 10 duplicate: 0.2336448598130841
---------------------


Between 2018 and August 2020
Total tiles: 3464
Total unique tiles: 106
% Tiles < 10 duplicate: 0.24528301886792453
---------------------


Between 2019 and August 2020
Total tiles: 1915
Total unique tiles: 101
% Tiles < 10 duplicate: 0.3465346534653465
---------------------


Between 2020 and August 2020
Total tiles: 914
Total unique tiles: 94
% Tiles < 10 duplicate: 0.574468085106383
---------------------




# Removing all duplicate tiles above 10 duplicates 

In [75]:
#Congo Basin Bounding Box
# search_bbox = BBox(bbox=[6.81,-5.27, 30, 5.5], crs=CRS.WGS84)

#Additional Params
bands = ['R10m/TCI']

#Republic of the Congo Bounding Box
search_bbox = BBox(bbox=[11.6395900000061374,-4.8205900000029942, 18.4909000000043306,3.6516899999987800], crs=CRS.WGS84)



search_time_interval = ('2019-01-01T00:00:00', '2020-12-31T23:59:59')
wfs_iterator = WebFeatureService(
    search_bbox,
    search_time_interval,
    data_source=DataSource.SENTINEL2_L2A,
    maxcc=.05,
    config=config
)
results = wfs_iterator.get_tiles()
df = pd.DataFrame(results, columns=['Tilename','Date','AmazonID'])
df2 = (df.groupby('Tilename').head(10))
output2 = list(df2.itertuples(index=False,name=None))
for tile in output2:
    tile_name, time, aws_index = tile
    data_folder = f'/Volumes/Lacie/zhenyadata/Project_Canopy_Data/Data/Sentinel_Data/ROC/L2A/SAFE/Full/2019'
    request = AwsTileRequest(
        tile=tile_name,
        time=time,
        bands = bands, 
        aws_index=aws_index,
        data_folder=data_folder,
        data_source=DataSource.SENTINEL2_L2A,
        safe_format = True
    )

    request.save_data(redownload=True)

/Users/user/.local/share/virtualenvs/data_collection-6QAt_VD6/lib/python3.7/site-packages/sentinelhub/aws.py:164: SHUserWarning: Products with baseline 02.14 are not officially supported in sentinelhub-py. If you notice any errors in naming structure of downloaded data please report an issue at https://github.com/sentinel-hub/sentinelhub-py/issues. Pull requests are also very appreciated
  warnings.warn(message, category=SHUserWarning)
/Users/user/.local/share/virtualenvs/data_collection-6QAt_VD6/lib/python3.7/site-packages/sentinelhub/aws.py:164: SHUserWarning: Products with baseline 02.14 are not officially supported in sentinelhub-py. If you notice any errors in naming structure of downloaded data please report an issue at https://github.com/sentinel-hub/sentinelhub-py/issues. Pull requests are also very appreciated
  warnings.warn(message, category=SHUserWarning)
/Users/user/.local/share/virtualenvs/data_collection-6QAt_VD6/lib/python3.7/site-packages/sentinelhub/aws.py:164: SHUser

# Test

In [18]:
data_folder = f'/Volumes/Lacie/zhenyadata/Project_Canopy_Data/Data/Sentinel_Data/Test/'
request = AwsTileRequest(
    tile="32MLE",
    time="2020-1-10",
    aws_index="0",
    data_folder=data_folder,
    data_source=DataSource.SENTINEL2_L1C,
    safe_format = True
)

request.save_data()

/Users/user/.local/share/virtualenvs/data_collection-6QAt_VD6/lib/python3.7/site-packages/sentinelhub/aws.py:164: SHUserWarning: Products with baseline 02.08 are not officially supported in sentinelhub-py. If you notice any errors in naming structure of downloaded data please report an issue at https://github.com/sentinel-hub/sentinelhub-py/issues. Pull requests are also very appreciated
  warnings.warn(message, category=SHUserWarning)
